In [1]:
import requests
from bs4 import BeautifulSoup
import psycopg2

class CountrySalary:
    def __init__(self, country, monthly_salary):
        self.country = country
        self.monthly_salary = monthly_salary
    
    def __repr__(self):
        return f"Country: {self.country}, Monthly Salary: {self.monthly_salary}"

# Função para verificar e tratar os salários
def process_salary(salary_text):
    if salary_text:
        # Remove símbolos de dólar e vírgulas
        salary_text = salary_text.replace('$', '').replace(',', '')
        try:
            salary = float(salary_text)  # Tenta converter para float
            return salary
        except ValueError:
            return None  # Se falhar na conversão, retorna None
    return None  # Caso o texto seja vazio ou inválido

# Função para extrair os dados da página da Wikipedia
# Função para extrair os dados da página da Wikipedia
def get_country_salaries(url):
    # Fazer a requisição para a página
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar a tabela com os dados
    table = soup.find('table', {'class': 'wikitable'})

    # Lista para armazenar os objetos CountrySalary
    country_salaries = []

    # Iterar pelas linhas da tabela
    for row in table.find_all('tr')[1:]:  # Ignorar a primeira linha (cabeçalho)
        cells = row.find_all('td')
        if len(cells) > 1:
            # Limpar o nome do país e remover o '*' se existir
            country = cells[0].get_text(strip=True).replace('*', '')  # Remover o asterisco
            salary_text = cells[2].get_text(strip=True)
            salary = process_salary(salary_text)
            # Cria um objeto CountrySalary e adiciona à lista
            country_salaries.append(CountrySalary(country, salary))

    return country_salaries



In [8]:




# Função para salvar os dados no PostgreSQL
def save_to_postgresql(country_salaries):
    # Configuração da conexão com o banco de dados PostgreSQL
    connection =  connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8" 
    )
    cursor = connection.cursor()
    
    # # Inserir os dados na tabela
    for country_salary in country_salaries:
        print(country_salary.country)
        # Vinculando o país com o banco de dados
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (country_salary.country,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            print(pais_id)
    #     # Tratamento de dados nulos antes de salvar
            cursor.execute("""
            INSERT INTO salary.country_salary (m_salary, country_id)
            VALUES (%s, %s);
            """, (country_salary.monthly_salary, pais_id))
            connection.commit()

        # Commit para garantir que os dados sejam salvos

    # # Fechar a conexão
    cursor.close()
    connection.close()

# URL de exemplo para salários mensais
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage'

# Obter os salários dos países
country_salaries = get_country_salaries(url)

# Exibir os dados extraídos para verificação
# for country_salary in country_salaries:
#     print(country_salary)

# Salvar os dados no PostgreSQL
save_to_postgresql(country_salaries)



Afghanistan
652230
Albania
28748
Algeria
2381741
Andorra
468
Angola
1246700
Antigua and Barbuda
442
Argentina
2780400
Armenia
29743
Australia
7692024
Austria
83871
Azerbaijan
86600
The Bahamas
Bahrain
765
Bangladesh
147570
Barbados
430
Belarus
207600
Belgium
30528
Belize
22966
Benin
112622
Bhutan
38394
Bolivia
1098581
Bosnia and Herzegovina
51209
Botswana
582000
Brazil
8515767
Brunei
5765
Bulgaria
110879
Burkina Faso
272967
Burundi
27834
Cambodia
181035
Cameroon
475442
Canada
9984670
Cape Verde
4033
Central African Republic
622984
Chad
1284000
Chile
756102
China
9706961
Colombia
1141748
Comoros
1862
Democratic Republic of the Congo
Republic of the Congo
342000
Costa Rica
51100
Côte d'Ivoire
Croatia
56594
Cuba
109884
Cyprus
9251
Czech Republic
Denmark
43094
Djibouti
23200
Dominica
751
Dominican Republic
48671
Ecuador
276841
Egypt
1002450
El Salvador
21041
Equatorial Guinea
28051
Eritrea
117600
Estonia
45227
Eswatini
17364
Ethiopia
1104300
Federated States of Micronesia
Fiji
18272
Finlan